In [233]:
import requests
import xml.etree.ElementTree as ET
import pandas as pd
from urllib import parse
import pymysql

In [234]:
key = "WRvGBkn9UEtw%2BAsg0tYo210Etxvb1QEcAX%2BwfWvOxVGYJkh1CNZ%2FY4QFa0r7j4bhT4NjPu7z1i1ck8ZgsKMt2Q%3D%3D"

In [235]:
url = "http://openapi.data.go.kr/openapi/service/rest/Covid19/getCovid19InfStateJson?serviceKey=WRvGBkn9UEtw%2BAsg0tYo210Etxvb1QEcAX%2BwfWvOxVGYJkh1CNZ%2FY4QFa0r7j4bhT4NjPu7z1i1ck8ZgsKMt2Q%3D%3D&pageNo=1&numOfRows=10&startCreateDt=20200101&endCreateDt=20210407"

## xml 파싱

In [236]:
r = requests.get(url)

In [237]:
r.url

'http://openapi.data.go.kr/openapi/service/rest/Covid19/getCovid19InfStateJson?serviceKey=WRvGBkn9UEtw%2BAsg0tYo210Etxvb1QEcAX%2BwfWvOxVGYJkh1CNZ%2FY4QFa0r7j4bhT4NjPu7z1i1ck8ZgsKMt2Q%3D%3D&pageNo=1&numOfRows=10&startCreateDt=20200101&endCreateDt=20210407'

In [238]:
root = ET.fromstring(r.text)

In [239]:
total = []
items = root.iter(tag='item')
for element in items:
    #print(element.find("accDefRate").text)
    covid_dict = {}
    for x in element:
        covid_dict[x.tag] = x.text
    total.append(covid_dict)

In [240]:
len(total)

433

In [241]:
df = pd.DataFrame(total)

In [242]:
df.head(2)

,accDefRate,accExamCnt,accExamCompCnt,careCnt,clearCnt,createDt,deathCnt,decideCnt,examCnt,resutlNegCnt,seq,stateDt,stateTime,updateDt
0,1.3545924266,7966167,7891525,7214,97928,2021-04-07 09:37:24.979,1756,106898,74642,7784627,473,20210407,00:00,null
1,1.3546056016,7921290,7842135,7115,97363,2021-04-06 09:31:57.498,1752,106230,79155,7735905,472,20210406,00:00,2021-04-06 19:12:35.607


* null로 들어오는 값 처리 

In [243]:
df = df.where(~(df == "null"), None)

* 사용할 컬럼 정렬

In [244]:
df.keys()

Index(['accDefRate', 'accExamCnt', 'accExamCompCnt', 'careCnt', 'clearCnt',
       'createDt', 'deathCnt', 'decideCnt', 'examCnt', 'resutlNegCnt', 'seq',
       'stateDt', 'stateTime', 'updateDt'],
      dtype='object')

In [245]:
col = ['createDt', 'decideCnt', 'deathCnt', 'accExamCompCnt' ]

In [246]:
분석대상 = df[col].copy()

In [247]:
분석대상.columns = ['생성일', '확진자', '사망자', '검사완료']

In [248]:
분석대상

,생성일,확진자,사망자,검사완료
0,2021-04-07 09:37:24.979,106898,1756,7891525
1,2021-04-06 09:31:57.498,106230,1752,7842135
2,2021-04-05 09:44:27.235,105752,1748,7788323
3,2021-04-04 09:45:05.147,105279,1744,7769677
4,2021-04-03 09:34:03.806,104736,1740,7754313
...,...,...,...,...
428,2020-02-03 21:26:59.59,0,0,NaN
429,2020-02-03 14:41:17.17,15,0,NaN
430,2020-02-03 12:22:49.49,2,2,NaN
431,2020-01-31 17:47:33.33,0,0,NaN


In [249]:
분석대상['날짜'] = 분석대상['생성일'].apply(lambda x : x.split()[0])

* 중복 데이터 확인

In [250]:
분석대상[분석대상['날짜'].duplicated()]

,생성일,확진자,사망자,검사완료,날짜
110,2020-12-19 9:39:07.07,47517,645,3479179,2020-12-19
120,2020-12-09 9:44:26.26,39417,556,3178141,2020-12-09
233,2020-08-18 9:12:23.23,15515,305,1665506,2020-08-18
429,2020-02-03 14:41:17.17,15,0,NaN,2020-02-03
430,2020-02-03 12:22:49.49,2,2,NaN,2020-02-03
432,2020-01-31 17:47:33.33,0,0,NaN,2020-01-31


In [251]:
분석대상.sort_values(by=['생성일'], inplace=True)

In [252]:
분석대상.shape

(433, 5)

nth()

In [253]:
분석대상 = 분석대상.groupby("날짜").nth(0).reset_index()

In [254]:
분석대상 = 분석대상.groupby("날짜").first().reset_index()

In [255]:
분석대상.query(" 날짜 == '2020-12-19'")

,날짜,생성일,확진자,사망자,검사완료
317,2020-12-19,2020-12-19 10:32:55.55,48563,659,3520008


* 파생변수 만들기 

In [256]:
분석대상.tail()

,날짜,생성일,확진자,사망자,검사완료
422,2021-04-03,2021-04-03 09:34:03.806,104736,1740,7754313
423,2021-04-04,2021-04-04 09:45:05.147,105279,1744,7769677
424,2021-04-05,2021-04-05 09:44:27.235,105752,1748,7788323
425,2021-04-06,2021-04-06 09:31:57.498,106230,1752,7842135
426,2021-04-07,2021-04-07 09:37:24.979,106898,1756,7891525


In [257]:
분석대상['확진자'] = 분석대상['확진자'].astype(int)
분석대상['확진자'] =분석대상['확진자'].diff(1)
분석대상['확진자'] =분석대상['확진자'].shift(-1)

In [258]:
분석대상['사망자'] = 분석대상['사망자'].astype(int)
분석대상['사망자'] =분석대상['사망자'].diff(1)
분석대상['사망자'] =분석대상['사망자'].shift(-1)

In [259]:
분석대상['검사완료'].fillna(0, inplace=True)
분석대상['검사완료'] = 분석대상['검사완료'].astype(int)
분석대상['검사완료'] =분석대상['검사완료'].diff(1)
분석대상['검사완료'] =분석대상['검사완료'].shift(-1)

In [260]:
df2 = 분석대상.copy()

## Database 연결 및 Create, Insert 

GRANT ALL PRIVILEGES ON *.* TO 'root'@'%' IDENTIFIED BY '12345'

In [261]:
try:
    con = pymysql.connect(host='13.125.69.29', user='root', port=3306, password='1q2w3e',   db='chart', charset='utf8')
    cur = con.cursor()
except Exception as e:
    print (e)   

In [262]:
for name in df2.keys():
    print(name , end=", ")

날짜, 생성일, 확진자, 사망자, 검사완료, 

In [263]:
'createDt', 'decideCnt', 'deathCnt', 'accExamCompCnt'

('createDt', 'decideCnt', 'deathCnt', 'accExamCompCnt')

In [264]:
create_table = """
    CREATE TABLE covid19 (
        `id`  int(10) NOT NULL AUTO_INCREMENT,
        `createDt`     DATETIME, 
        `decideCnt`    INT(4),
        `deathCnt`     INT(4),
        `accExamCompCnt` INT(4),
        PRIMARY KEY (`id`)
    ) 
"""

In [265]:
cur.execute(create_table)

0

In [266]:
con.commit()

In [267]:
sql = """INSERT INTO covid19 (createDt, decideCnt, deathCnt, accExamCompCnt) VALUES
         (%s, %s, %s, %s)
"""

In [268]:
df2

,날짜,생성일,확진자,사망자,검사완료
0,2020-01-31,2020-01-31 17:47:33.33,2.0,2.0,0.0
1,2020-02-03,2020-02-03 12:22:49.49,17.0,-2.0,0.0
2,2020-02-05,2020-02-05 20:05:40.40,4.0,0.0,0.0
3,2020-02-06,2020-02-06 9:09:49.49,1.0,0.0,0.0
4,2020-02-07,2020-02-07 17:20:45.45,0.0,0.0,0.0
...,...,...,...,...,...
422,2021-04-03,2021-04-03 09:34:03.806,543.0,4.0,15364.0
423,2021-04-04,2021-04-04 09:45:05.147,473.0,4.0,18646.0
424,2021-04-05,2021-04-05 09:44:27.235,478.0,4.0,53812.0
425,2021-04-06,2021-04-06 09:31:57.498,668.0,4.0,49390.0


In [269]:
import numpy as np

In [270]:
df2 = df2.where(pd.notnull(df2), None)

In [271]:
df2.drop(columns=['생성일'], inplace=True)

In [272]:
for row in df2.iterrows():
    cur.execute(sql, row[1].tolist())
    
con.commit()